# Workflow to Validate NISAR L2 Secular Displacement Requirement

**Original code authored by:** David Bekaert, Heresh Fattahi, Eric Fielding, and Zhang Yunjun with 
Extensive modifications by Adrian Borsa and Amy Whetter and other NISAR team members 2022

**Updated for OPERA requirements by Simran Sangha, Marin Govorcin, and Al Handwerger**


<div class="alert alert-warning">
Both the initial setup (<b>Prep A</b> section) and download of the data (<b>Prep B</b> section) should be run at the start of the notebook. And all subsequent sections NEED to be run in order.
</div>



## Define CalVal Site 

In [ ]:
### Choose a site from the 'sites' dictionary found 2 cells down
## If your study area is not defined, add a new dictionary entry as appropriate and provide a unique site keyname
site = 'CentralValleyD144_4yr'

# set path to aria-tools/MintPy/FRInGE output (if already generated)
work_dir = ''

In [ ]:
# define credentials
earthdata_user = ''
earthdata_password = ''
opentopography_api_key = ''

In [ ]:
### Define list of requirements
## Static for OPERA Cal/Val requirements, do not touch!

findMax = 'true' # set to 'true' if you want to find the maximum threshold, set to 'false' if you want to find the minimum threshold

# Define secular requirements
secular_gnss_rqmt = 3  # mm/yr for 3 years of data over length scales of 0.1-50 km
gnss_dist_rqmt = [0.1, 50.0]  # km
secular_insar_rqmt = 3  # mm/yr
insar_dist_rqmt = [0.1, 50.0]  # km

# Define temporal sampling requirement
insar_sampling = 12 # days
insar_sampling_percentage = 80 # percentage of acquitions at 12 day sampling (insar_sampling) or better
insar_timespan_requirement = 4 # years

# Define spatial coherence threshold (necessary to reject poor quality, long temporal baseline pairs)
coherenceBased_parm = 'yes'
minCoherence_parm = 0.4

# Set mask file
maskFile = 'maskTempCoh.h5' # maskTempCoh.h5 maskConnComp.h5 waterMask.h5 (maskConnComp.h5 is very conservative)

In [ ]:
### List of CalVal Sites:
## Set maskWater flag to True if you are not working with AO restricted to within a continental interior
'''
Set NISAR calval sites:
    CentralValleyD144  : Central Valley - Descending track
    CentralValleyA137  : Central Valley - Ascending track

ARIA & MintPy parameters:
    calval_location : name
    download_region : download box in S,N,W,E format
    analysis_region : analysis box in S,N,W,E format (must be within download_region)
    reference_lalo : latitute,longitude in geographic coordinates (default: auto)
    download_start_date : download start date as YYYMMDD  
    download_end_date   : download end date as YYYMMDD
    sentinel_track : sentinel track to download
    gps_ref_site_name : Name of the GPS site for InSAR re-referencing
    tempBaseMax' : maximum number of days, 'don't use interferograms longer than this value 
    ifgExcludeList : default is not to exclude any interferograms
    earthquakeDate' : specify date of EQ step/volcanic eruption. if not applicable, specify is auto which is the default
    maskWater' :  interior locations don't need to mask water
'''
sites = {
    ##########  CENTRAL VALLEY descending ##############
    'CentralValleyD144_4yr' : {'calval_location' : 'CentralValleyD144_4yr',
            'download_region' : '"36.18 36.26 -119.91 -119.77"', # download box in S,N,W,E format
            'analysis_region' : '"35.77 36.75 -120.61 -118.06"', # analysis box in S,N,W,E format (must be within download_region)
            'reference_lalo' : 'auto',
            'download_start_date' : '20180101',
            'download_end_date' : '20220101',
            'sentinel_track' : '144',
            'gps_ref_site_name' : 'P056', # reference site for this area
            'tempBaseMax' : 'auto',
            'ifgExcludeList' : 'auto',
            'earthquakeDate' : '20190705', # time for Ridgecrest EQ. Change to `auto` to toggle off if there is no deformation event
            'maskWater' : False,
            'reference_mask' : 'auto',
            'use_staged_data': False, # option to control the use of pre-staged data; [False/True]
            'use_mintpy': True, # specify use of MintPy (applicable to all cases aside for FRInGE); [False/True]
            'vmin' : -100,
            'vmax' : 100},
    ##########  CENTRAL VALLEY ascending ##############
    'CentralValleyA137_4yr' : {'calval_location' : 'CentralValleyA137_4yr',
            'download_region' : '"36.18 36.26 -119.91 -119.77"', # download box in S,N,W,E format
            'analysis_region' : '"35.77 36.75 -120.61 -118.06"', # analysis box in S,N,W,E format (must be within download_region)
            'reference_lalo' : 'auto',
            'download_start_date' : '20180101',
            'download_end_date' : '20220101',
            'sentinel_track' : '137',
            'gps_ref_site_name' : 'P056', # reference site for this area
            'tempBaseMax' : 'auto',
            'ifgExcludeList' : 'auto',
            'earthquakeDate' : '20190705', # time for Ridgecrest EQ. Change to `auto` to toggle off if there is no deformation event
            'maskWater' : False,
            'reference_mask' : 'auto',
            'use_staged_data': False, # option to control the use of pre-staged data; [False/True]
            'use_mintpy': True, # specify use of MintPy (applicable to all cases aside for FRInGE); [False/True]
            'vmin' : -100,
            'vmax' : 100},
    ##########  Mojave descending ##############
    'MojaveD173_4year': {'calval_location' : 'MojaveD173_4year',
            'download_region' : '"34.5 35.6 -116.62 -114.39"', # download box in S,N,W,E format
            'analysis_region' : '"34.66 35.60 -116.62 -114.39"', # analysis box in S,N,W,E format (must be within download_region)
            'reference_lalo' : '35.20495,-115.81229',
            'download_start_date' : '20180101',
            'download_end_date' : '20220101',
            'sentinel_track' : '173',
            'gps_ref_site_name' : 'P619', # reference site for this area
            'tempBaseMax' : 'auto',
            'ifgExcludeList' : 'auto',
            'earthquakeDate' : '20190705', # time for Ridgecrest EQ. Change to `auto` to toggle off if there is no deformation event
            'maskWater' : False,
            'reference_mask' : 'auto',
            'use_staged_data': False, # option to control the use of pre-staged data; [False/True]
            'use_mintpy': True, # specify use of MintPy (applicable to all cases aside for FRInGE); [False/True]
            'vmin' : -20,
            'vmax' : 20},
    ##########  Mojave descending ##############
    'MojaveA166_4year': {'calval_location' : 'MojaveA166_4year',
            'download_region' : '"34.5 35.6 -116.62 -114.39"', # download box in S,N,W,E format
            'analysis_region' : '"34.66 35.60 -116.62 -114.39"', # analysis box in S,N,W,E format (must be within download_region)
            'reference_lalo' : '35.20495,-115.81229',
            'download_start_date' : '20180101',
            'download_end_date' : '20220101',
            'sentinel_track' : '166',
            'gps_ref_site_name' : 'P611',
            'tempBaseMax' : 'auto',
            'ifgExcludeList' : 'auto',
            'earthquakeDate' : '20190705', # time for Ridgecrest EQ. Change to `auto` to toggle off if there is no deformation event
            'maskWater' : False,
            'reference_mask' : 'no',
            'use_staged_data': False, # option to control the use of pre-staged data; [False/True]
            'use_mintpy': True, # specify use of MintPy (applicable to all cases aside for FRInGE); [False/True]
            'vmin' : -20,
            'vmax' : 20},
    ##########  Descending ##############
    'HoustonD143' : {'calval_location' : 'HoustonD143',
            'download_region' : '"29.120047 29.935538 -95.864902 -94.555116"', # download box in S,N,W,E format
            'analysis_region' : '"29.120047 29.935538 -95.864902 -94.555116"', # analysis box in S,N,W,E format (must be within download_region)
            'reference_lalo' : '29.757039,-95.355618',
            'download_start_date' : '20160101',
            'download_end_date' : '20200101',         
            'sentinel_track' : '143',
            'gps_ref_site_name' : 'UH01', # reference site for this area
            'tempBaseMax' : 'auto',
            'ifgExcludeList' : 'auto',
            'earthquakeDate' : 'auto', # time for Ridgecrest EQ. Change to `auto` to toggle off if there is no deformation event
            'use_staged_data': False, # option to control the use of pre-staged data; [False/True]
            'use_mintpy': True, # specify use of MintPy (applicable to all cases aside for FRInGE); [False/True]
            'maskWater' : True},
    ##########  Ascending ##############
    'HoustonA034' : {'calval_location' : 'HoustonA034',
            'download_region' : '"29.120047 29.935538 -95.864902 -94.555116"', # download box in S,N,W,E format
            'analysis_region' : '"29.120047 29.935538 -95.864902 -94.555116"', # analysis box in S,N,W,E format (must be within download_region)
            'reference_lalo' : '29.757039,-95.355618',
            'download_start_date' : '20160101',
            'download_end_date' : '20200101',
            'sentinel_track' : '34',
            'gps_ref_site_name' : 'UH01', # reference site for this area
            'tempBaseMax' : 'auto',
            'ifgExcludeList' : 'auto',
            'earthquakeDate' : 'auto', # time for Ridgecrest EQ. Change to `auto` to toggle off if there is no deformation event
            'use_staged_data': False, # option to control the use of pre-staged data; [False/True]
            'use_mintpy': True, # specify use of MintPy (applicable to all cases aside for FRInGE); [False/True]
            'maskWater' : True}
}
secular_available_sites = list(sites.keys())

## Table of Contents:
<a id='secular_TOC'></a>

<hr/>

[**Prep A. Environment Setup**](#secular_prep_a)

[**Prep B. Data Staging**](#secular_prep_b)

[**1. Generate Interferogram Stack**](#secular_gen_ifg)
- [1.1.  Crop Interferograms](#secular_crop_ifg)

[**2. Generation of Time Series from Interferograms**](#secular_gen_ts)
- [2.1. Set Up MintPy Configuration file](#secular_setup_config)
- [2.2. Load Data into MintPy](#secular_load_data)
- [2.3. Validate/Modify Interferogram Network](#secular_validate_network)
- [2.4. Generate Quality Control Mask](#secular_generate_mask)
- [2.5. Reference Interferograms To Common Lat/Lon](#secular_common_latlon)
- [2.6. Invert for SBAS Line-of-Sight Timeseries](#secular_invert_SBAS)

[**3. Optional Corrections**](#secular_opt_correction)
- [3.1. Solid Earth Tide Correction](#secular_solid_earth)
- [3.2. Tropospheric Delay Correction](#secular_tropo_corr)
- [3.3. Phase Deramping ](#secular_phase_deramp)
- [3.4. Topographic Residual Correction ](#secular_topo_corr) 

[**4. Estimate InSAR and GNSS Velocities**](#secular_decomp_ts)
- [4.1. Estimate InSAR LOS Velocities](#secular_insar_vel1)
- [4.2. Find Collocated GNSS Stations](#secular_co_gps)  
- [4.3. Get GNSS Position Time Series](#secular_gps_ts) 
- [4.4. Make GNSS LOS Velocities](#secular_gps_los)
- [4.5. Re-Reference GNSS and InSAR Velocities](#secular_gps_insar)

[**5. NISAR Validation Approach 1: GNSS-InSAR Direct Comparison**](#secular_nisar_validation)
- [5.1. Make Velocity Residuals at GNSS Locations](#secular_make_vel)
- [5.2. Make Double-differenced Velocity Residuals](#secular_make_velres)
- [5.3. Secular Requirement Validation: Method 1](#secular_valid_method1)

[**6. NISAR Validation Approach 2: InSAR-only Structure Function**](#secular_nisar_validation2)
- [6.1. Read Array and Mask Pixels with no Data](#secular_array_mask)
- [6.2. Randomly Sample Pixels and Pair Them Up with Option to Remove Trend](#secular_remove_trend)
- [6.3. Amplitude vs. Distance of Relative Measurements (pair differences)](#secular_M2ampvsdist2)
- [6.4. Bin Sample Pairs by Distance Bin and Calculate Statistics](#secular_M2RelMeasTable)

[**Appendix: Supplementary Comparisons and Plots**](#secular_appendix1)
- [A.1. Compare Raw Velocities](#secular_compare_raw)
- [A.2. Plot Velocity Residuals](#secular_plot_vel)
- [A.3. Plot Double-differenced Residuals](#secular_plot_velres)
- [A.4. GPS Position Plot](#secular_appendix_gps)

<hr/>

<a id='secular_prep_a'></a>
## Prep A. Environment Setup
Setup your environment for processing data

In [ ]:
#Load Packages
import glob
import os
import subprocess
from datetime import datetime as dt
import math
from pathlib import Path

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from mintpy.objects import gps, timeseries
from mintpy.objects.gps import search_gps, GPS
from mintpy.smallbaselineApp import TimeSeriesAnalysis
from mintpy.utils import ptime, readfile, utils as ut
from mintpy.cli import view, plot_network
from scipy import signal

from solid_utils.sampling import load_geo, samp_pair, profile_samples, haversine_distance

from copy import deepcopy

#Set Global Plot Parameters
plt.rcParams.update({'font.size': 12})

################# Set Directories ##########################################
print('\nCurrent directory:',os.getcwd())

work_dir = Path(work_dir)

print("Work directory:", work_dir)
work_dir.mkdir(parents=True, exist_ok=True)
# Change to Workdir
os.chdir(work_dir)

gunw_dir = work_dir/'products'
gunw_dir.mkdir(parents=True, exist_ok=True)
print("   GUNW    dir:", gunw_dir) 

mintpy_dir = work_dir/'MintPy' 
mintpy_dir.mkdir(parents=True, exist_ok=True)
print("   MintPy  dir:", mintpy_dir)
### Change to MintPy workdir
os.chdir(mintpy_dir)
vel_file = os.path.join(mintpy_dir, 'velocity.h5')
msk_file = os.path.join(mintpy_dir, maskFile)  # maskTempCoh.h5 maskConnComp.h5 waterMask.h5

if site not in secular_available_sites:
    msg = '\nSelected site not available! Please select one of the following sites:: \n{}'.format(secular_available_sites)
    raise Exception(msg)
else:
    print('\nSelected site: {}'.format(site))
    for key, value in sites[site].items():
        print('   '+ key, ' : ', value)

<a id='secular_prep_b'></a>
## Prep B. Data Staging

In this initial processing step, all the necessary Level-2 unwrapped interferogram products are gathered, organized and reduced to a common grid for analysis with MintPy. Ascending and descending stacks of nearest-neighbor and skip-1 interferograms will be prepared for independent analysis. 

In [ ]:
######### DO NOT CHANGE LINES BELOW ########

##################### 1. Download (Aria) Interferograms from ASF ################
os.chdir(work_dir)
# do not re-download products
if os.path.exists('products'):
    if not os.listdir('products'):
        print('NEEDED To Download ARIA GUNWs: \n Link to create account : https://urs.earthdata.nasa.gov/')
        print('NEEDED To Download DEMs: \n Link to create account : https://portal.opentopography.org/login')
        if os.path.exists('~/.topoapi'): # if OpenTopo API key already installed
            print('OpenTopo API key appears to be installed, using that')
        else:
            print('API key location: My Account > myOpenTopo Authorizations and API Key > Request API key')
            #opentopography_api_key = input('Please type your OpenTopo API key:')

        ######################## USE ARIA-TOOLS TO DOWNLOAD GUNW ########################
        '''
        REFERENCE: https://github.com/aria-tools/ARIA-tools
        '''
        aria_download = '''ariaDownload.py -b {bbox} -u {user} -p {password} -s {start}  -e {end} -t {track} -o Count'''

        ###############################################################################
        print('CalVal site {}'.format(site))
        print('  Searching for available GUNW products:\n')

        command = aria_download.format(bbox = sites[site]['download_region'],
                                    start = sites[site]['download_start_date'],
                                    end = sites[site]['download_end_date'],
                                    track = sites[site]['sentinel_track'],
                                    user = earthdata_user,
                                    password = earthdata_password)
        
        process = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text = True, shell = True)
        print(process.stdout)

        ############## Download GUNW ##################
        print("Start downloading GUNW files ...")
        process = subprocess.run(command.split(' -o')[0], stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, shell=True)
        # Missing progressbar
        print("Downloaded {} GUNW files in: {}\n".format(len([(x) for x in os.listdir(gunw_dir) if x.endswith('.nc')]), gunw_dir))

        ############## DO little CLEANING ###########
        data_to_clean = ["avg_rates.csv", "ASFDataDload0.py", "AvgDlSpeed.png", "error.log"]

        for i, file in enumerate(data_to_clean):
            print('Cleaning unnecessary data {} in {}'.format(file, gunw_dir))
            (gunw_dir/file).unlink(missing_ok=True)

        #Delete error log file from workdir
        print('Cleaning unnecessary data error.log in {}'.format(work_dir))
        (work_dir/"error.log").unlink(missing_ok=True)


<a id='secular_gen_ifg'></a>
# 1. Generate Interferogram Stack

InSAR time series (i.e., the unfiltered displacement of each pixel vs. time) are estimated from a processed InSAR stack from Section 3.1 (either ascending or descending) using a variant of the small baseline subset (SBAS) approach and then parameterized using the approach described in Section 4. This step uses tools available in the MintPy software package (REF), which provides both SBAS time series and model-based time series parameterization. Recent results on InSAR closure phase and “fading signal” recommend the of use all available interferograms to avoid systematic bias (_Ansari et al._, 2020; _Zheng Y.J. et al._, 2022). As we expect high-quality orbital control for NISAR, we anticipate that the interferogram stack will typically include all nearest-neighbor (i.e., ~12-day pairs) and skip-1 interferograms, which will be the minimum inputs into the SBAS generation step.

We use the open-source ARIA-tools package to download processed L2 interferograms over selected cal/val regions from the Alaska Satellite Facility archive and to stitch/crop the frame-based NISAR GUNW products to stacks that can be directly ingested into MintPy for time-series processing. ARIA-tools uses a phase-minimization approach in the product overlap region to stitch the unwrapped and ionospheric phase, a mosaicing approach for coherence and amplitude, and extracts the geometric information from the 3D data cubes through a mosaicking of the 3D datacubes and subsequent intersection with a DEM. ARIA has been used to pre-process NISAR beta products derived from Sentinel-1 which have revealed interseismic deformation and creep along the San Andreas Fault system, along with subsidence, landsliding, and other signals. 

We use MintPy to validate and modify the InSAR stack, removing interferograms that do not meet minimum coherence criteria, generating a quality control mask for the purpose of identifying noisy pixels within the stack, and referencing estimated deformation to a common location in all interferograms.

<a id='secular_crop_ifg'></a>
## 1.1. Crop Interferograms

In [ ]:
# Crop Interferograms to Analysis Region
mask_file = 'auto'
if not sites[site]['use_staged_data']:
    ###########################################################################################################
    # Set up ARIA product and mask data with GSHHS water mask:
    '''
    REQUIRED: Acquire API key to access/download DEMs

    Follow instructions listed here to generate and access API key through OpenTopography:
    https://opentopography.org/blog/introducing-api-keys-access-opentopography-global-datasets.
    '''
    
    if not os.path.exists(work_dir/'stack'):
        if not os.path.exists('~/.topoapi'): # if OpenTopo API key not already installed
            if 'opentopography_api_key' not in locals():
                print('API key location: My Account > myOpenTopo Authorizations and API Key > Request API key')
                #opentopography_api_key = input('Please type your OpenTopo API key:')
            os.system('echo "{api_key}" > ~/.topoapi; chmod 600 ~/.topoapi'.format(api_key = str(opentopography_api_key)))
        print('Preparing GUNWs for MintPY....')
        if sites[site]['maskWater']:
            mask_file = '../mask/watermask.msk'
            command = 'ariaTSsetup.py -f "products/*.nc" -b ' + sites[site]['analysis_region'] + ' --mask Download  --croptounion --verbose' # slow
        else: # skip slow mask download when we don't need to mask water
            command = 'ariaTSsetup.py -f "products/*.nc" -b ' + sites[site]['analysis_region'] + ' --croptounion --verbose'

        ################################## CROP & PREPARE STACK ###################################################
        print(command)
        result = subprocess.run(command, stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT, text=True, shell=True)
    print('Finish preparing GUNWs for MintPy!!')

####################################################################
### Change to MintPy workdir
os.chdir(mintpy_dir)

<a id='secular_gen_ts'></a>
# 2. Generation of Time Series from Interferograms

InSAR time series (i.e., the unfiltered displacement of each pixel vs. time) are estimated from a processed InSAR stack from Section 3.1 (either ascending or descending) using a variant of the small baseline subset (SBAS) approach and then parameterized using the approach described in Section 4. This step uses tools available in the MintPy software package (REF), which provides both SBAS time series and model-based time series parameterization. Recent results on InSAR closure phase and “fading signal” recommend the of use all available interferograms to avoid systematic bias (_Ansari et al._, 2020; _Zheng Y.J. et al._, 2022). As we expect high-quality orbital control for NISAR, we anticipate that the interferogram stack will typically include all nearest-neighbor (i.e., ~12-day pairs) and skip-1 interferograms, which will be the minimum inputs into the SBAS generation step.

We use the open-source ARIA-tools package to download processed L2 interferograms over selected cal/val regions from the Alaska Satellite Facility archive and to stitch/crop the frame-based NISAR GUNW products to stacks that can be directly ingested into MintPy for time-series processing. ARIA-tools uses a phase-minimization approach in the product overlap region to stitch the unwrapped and ionospheric phase, a mosaicing approach for coherence and amplitude, and extracts the geometric information from the 3D data cubes through a mosaicking of the 3D datacubes and subsequent intersection with a DEM. ARIA has been used to pre-process NISAR beta products derived from Sentinel-1 which have revealed interseismic deformation and creep along the San Andreas Fault system, along with subsidence, landsliding, and other signals. 

We use MintPy to validate and modify the InSAR stack, removing interferograms that do not meet minimum coherence criteria, generating a quality control mask for the purpose of identifying noisy pixels within the stack, and referencing estimated deformation to a common location in all interferograms.

<a id='secular_setup_config'></a>
## 2.1. Set Up MintPy Configuration file


The default processing parameters for MintPy's **smallbaselineApp.py** need to be modified by including the following lines in config_file (which must be manually created and placed into mint_dir):

- mintpy.load.processor      = aria
- mintpy.load.unwFile        = ../stack/unwrapStack.vrt
- mintpy.load.corFile        = ../stack/cohStack.vrt
- mintpy.load.connCompFile   = ../stack/connCompStack.vrt
- mintpy.load.demFile        = ../DEM/SRTM_3arcsec.dem
- mintpy.load.incAngleFile   = ../incidenceAngle/{download_start_date}_{download_edn_date}.vrt
- mintpy.load.azAngleFile    = ../azimuthAngle/{download_start_date}_{download_edn_date}.vrt
- mintpy.load.waterMaskFile  = ../mask/watermask.msk
- mintpy.reference.lalo      = auto, or somewhere in your bounding box
- mintpy.topographicResidual.pixelwiseGeometry = no
- mintpy.troposphericDelay.method              = no
- mintpy.topographicResidual                   = no

In [ ]:
config_file = Path(mintpy_dir)/(sites[site]['calval_location'] + '.cfg')

dem_file = glob.glob('../DEM/*dem')[0]

if not sites[site]['use_staged_data'] and sites[site]['use_mintpy'] is True:
    ####################################################################
    ### Write smallbaseline.py config file
    config_file_content = """
    mintpy.load.processor = aria
    mintpy.compute.numWorker = auto
    mintpy.load.unwFile = ../stack/unwrapStack.vrt
    mintpy.load.corFile = ../stack/cohStack.vrt
    mintpy.load.connCompFile = ../stack/connCompStack.vrt
    mintpy.load.demFile = {dem_file}
    mintpy.load.incAngleFile = ../incidenceAngle/*.vrt
    mintpy.load.azAngleFile = ../azimuthAngle/*.vrt
    mintpy.load.waterMaskFile = {mask_file}
    mintpy.networkInversion.minTempCoh = 0.5
    mintpy.topographicResidual.pixelwiseGeometry = no
    mintpy.troposphericDelay.method = no
    mintpy.topographicResidual = no
    mintpy.network.tempBaseMax = {tempmax}
    mintpy.network.startDate = {startdatenet}
    mintpy.network.endDate = {enddatenet}
    mintpy.network.coherenceBased = {coherenceBased}
    mintpy.network.minCoherence = {minCoherence}
    mintpy.timeFunc.startDate = {startdatevel}
    mintpy.timeFunc.endDate = {enddatevel}
    mintpy.timeFunc.stepDate = auto
    mintpy.reference.lalo = {reference_lalo}
    mintpy.reference.maskFile = {reference_mask}
    mintpy.network.excludeIfgIndex = {excludeIfg}""".format(dem_file = dem_file,
                                                            mask_file = mask_file,
                                                            tempmax=sites[site]['tempBaseMax'],
                                                            excludeIfg=sites[site]['ifgExcludeList'],
                                                            startdatenet=sites[site]['download_start_date'],
                                                            enddatenet=sites[site]['download_end_date'],
                                                            coherenceBased = coherenceBased_parm,
                                                            minCoherence = minCoherence_parm,
                                                            startdatevel=sites[site]['download_start_date'],
                                                            enddatevel=sites[site]['download_end_date'],
                                                            stepdate=sites[site]['earthquakeDate'],
                                                            reference_lalo=sites[site]['reference_lalo'],
                                                            reference_mask=sites[site]['reference_mask'])

    config_file.write_text(config_file_content)

print('MintPy config file:\n    {}:'.format(config_file))
print(config_file.read_text())

<a id='secular_load_data'></a>
## 2.2. Load Data into MintPy

The output of this step is an "inputs" directory in 'calval_directory/calval_location/MintPy/" containing two HDF5 files:
- ifgramStack.h5: This file contains 6 dataset cubes (e.g. unwrapped phase, coherence, connected components etc.) and multiple metadata
- geometryGeo.h5: This file contains geometrical datasets (e.g., incidence/azimuth angle, masks, etc.)

In [ ]:
if sites[site]['use_mintpy']:
    command = 'smallbaselineApp.py ' + str(config_file) + ' --dostep load_data'
    process = subprocess.run(command, shell=True)
    print('Mintpy input files:')
    [x for x in os.listdir('inputs') if x.endswith('.h5')]

<a id='secular_validate_network'></a>
## 2.3. Validate/Modify Interferogram Network

Add additional parameters to config_file in order to remove selected interferograms, change minimum coherence, etc.

In [ ]:
if sites[site]['use_mintpy']:
    command = 'smallbaselineApp.py ' + str(config_file) + ' --dostep modify_network'
    process = subprocess.run(command, shell=True)

In [ ]:
if sites[site]['use_mintpy']:
    try:
        plot_network.main(['inputs/ifgramStack.h5'])
    except ValueError: # circumvents 0 bperp (https://github.com/insarlab/MintPy/issues/281)
        pass

<a id='secular_generate_mask'></a>
## 2.4. Generate Quality Control Mask

Mask files can be can be used to mask pixels in the time-series processing. Below we generate a mask file based on the connected components, which is a metric for unwrapping quality.

In [ ]:
if sites[site]['use_mintpy']:
    command='generate_mask.py inputs/ifgramStack.h5  --nonzero  -o maskConnComp.h5  --update'
    process = subprocess.run(command, shell=True)

In [ ]:
if sites[site]['use_mintpy']:
    msk_file_concomp = os.path.join(mintpy_dir, 'maskConnComp.h5')
    view.main([msk_file_concomp, 'mask'])

<a id='secular_common_latlon'></a>
## 2.5. Reference Interferograms To Common Lat/Lon

In [ ]:
if sites[site]['use_mintpy']:
    command = 'smallbaselineApp.py ' + str(config_file) + ' --dostep reference_point'
    process = subprocess.run(command, shell=True)
    os.system('info.py inputs/ifgramStack.h5 | egrep "REF_"');

<a id='secular_invert_SBAS'></a>
## 2.6. Invert for SBAS Line-of-Sight Timeseries

In [ ]:
if sites[site]['use_mintpy']:
    command = 'smallbaselineApp.py ' + str(config_file) + ' --dostep invert_network'
    process = subprocess.run(command, shell=True)

In [ ]:
# grab the time-series file used for time function estimation given the template setup
template = readfile.read_template(os.path.join(mintpy_dir, 'smallbaselineApp.cfg'))
template = ut.check_template_auto_value(template)
insar_ts_file = TimeSeriesAnalysis.get_timeseries_filename(template, mintpy_dir)['velocity']['input']

# Get date list
date_list = timeseries(insar_ts_file).get_date_list()
num_date = len(date_list)
date0, date1 = date_list[0], date_list[-1]
insar_dates = ptime.date_list2vector(date_list)[0]

# Check temporal sampling
insar_sampling_arr = []
for i in range(len(insar_dates)-1):
    diff = (insar_dates[i+1] - insar_dates[i]).days
    insar_sampling_arr.append(diff)

count = 0
for i in insar_sampling_arr:
    if i <= insar_sampling:
        count += 1

percentage = (count / len(insar_sampling_arr)) * 100
timespan_of_insar=(insar_dates[len(insar_dates)-1]-insar_dates[0]).days /365.25

# Overall pass/fail criterion
if percentage >= insar_sampling_percentage:
    print(f'This velocity dataset ({percentage}%) passes the temporal sampling requirement ({insar_sampling_percentage}%)')
else:
    print(f'This velocity dataset ({percentage}%) does NOT pass the temporal sampling requirement ({insar_sampling_percentage}%)')

if timespan_of_insar >= insar_timespan_requirement:
    print(f'This velocity dataset ({timespan_of_insar} years) passes the timespan requirement ({insar_timespan_requirement} years)')
else:
    print(f'This velocity dataset ({timespan_of_insar} years) does NOT pass the timespan requirement ({insar_timespan_requirement } years)')
    
       


<a id='secular_tropo_corr'></a>
## 3.2. Tropospheric Delay Correction

Optional atmospheric correction utilizes the PyAPS (Jolivet et al., 2011, Jolivet and Agram, 2012) module within GIAnT (or eventually a merged replacement for GIAnT and MintPy). PyAPS is well documented, maintained and can be freely downloaded. PyAPS is included in GIAnT distribution). PyAPS currently includes support for ECMWF’s ERA-Interim, NOAA’s NARR and NASA’s MERRA weather models. A final selection of atmospheric models to be used for operational NISAR processing will be done during Phase C.

[T]ropospheric delay maps are produced from atmospheric data provided by Global Atmospheric Models. This method aims to correct differential atmospheric delay correlated with the topography in interferometric phase measurements. Global Atmospheric Models (hereafter GAMs)... provide estimates of the air temperature, the atmospheric pressure and the humidity as a function of elevation on a coarse resolution latitude/longitude grid. In PyAPS, we use this 3D distribution of atmospheric variables to determine the atmospheric phase delay on each pixel of each interferogram.

The absolute atmospheric delay is computed at each SAR acquisition date. For a pixel a_i at an elevation z at acquisition date i, the four surrounding grid points are selected and the delays for their respective elevations are computed. The resulting delay at the pixel a_i is then the bilinear interpolation between the delays at the four grid points. Finally, we combine the absolute delay maps of the InSAR partner images to produce the differential delay maps used to correct the interferograms.

[MintPy provides functionality for this correction.]

In [ ]:
do_tropo_correction = False
########################################################################
'''
REFERENCE : https://github.com/insarlab/pyaps#2-account-setup-for-era5
Read Section 2 for ERA5 [link above] to create an account on the CDS website.
'''

if sites[site]['use_mintpy']:
    if do_tropo_correction:
        if not sites[site]['use_staged_data'] and not os.path.exists(Path.home()/'.cdsapirc'):
            print('NEEDED to download ERA5, link: https://cds.climate.copernicus.eu/user/register')
            #UID = input('Please type your CDS_UID:')
            #CDS_API = input('Please type your CDS_API:')
            
            cds_tmp = '''url: https://cds.climate.copernicus.eu/api/v2
            key: {UID}:{CDS_API}'''.format(UID=UID, CDS_API=CDS_API)
            os.system('echo "{cds_tmp}" > ~/.cdsapirc; chmod 600 ~/.cdsapirc'.format(cds_tmp = str(cds_tmp)))
        
        command = 'smallbaselineApp.py ' + str(config_file) + ' --dostep correct_troposphere'
        process = subprocess.run(command, shell=True)
        
        view.main(['inputs/ERA5.h5'])
        timeseries_filename = 'timeseries_ERA5.h5'
    else:
        timeseries_filename = 'timeseries.h5'

<a id='secular_phase_deramp'></a>
## 3.3. Phase Deramping


In [ ]:
if sites[site]['use_mintpy']:
    command = 'smallbaselineApp.py ' + str(config_file) + ' --dostep deramp'
    process = subprocess.run(command, shell=True)

<a id='secular_topo_corr'></a>
## 3.4. Topographic Residual Correction 

[MintPy provides functionality for this correction.]

In [ ]:
if sites[site]['use_mintpy']:
    command = 'smallbaselineApp.py ' + str(config_file) + ' --dostep correct_topography'
    process = subprocess.run(command, shell=True)

<a id='secular_decomp_ts'></a>
# 4. Estimate InSAR and GNSS Velocities
The approach that will be used for the generation of NISAR L3 products for Requirements 660 and 663 allows for an explicit inclusion of key basis functions (e.g., Heaviside functions, secular rate, etc.) in the InSAR inversion. Modifications to this algorithm may be identified and implemented in response to NISAR Phase C activities. 

<a id='secular_insar_vel1'></a>
## 4.1. Estimate InSAR LOS Velocities

Given a time series of InSAR LOS displacements, the observations for a given pixel, $U(t)$, can be parameterized as:

$$U(t) = a \;+\; vt \;+\; c_1 cos (\omega_1t - \phi_{1,}) \;+\; c_2 cos (\omega_2t - \phi_2) \;+\; \sum_{j=1}^{N_{eq}} \left( h_j+f_j F_j (t-t_j) \right)H(t - t_j) \;+\; \frac{B_\perp (t)}{R sin \theta}\delta z \;+\; residual$$ 

which includes a constant offset $(a)$, velocity $(v)$, and amplitudes $(c_j)$ and phases $(\phi_j)$ of annual $(\omega_1)$ and semiannual $(\omega_2)$ sinusoidal terms.  Where needed we can include additional complexity, such as coseismic and postseismic processes parameterized by Heaviside (step) functions $H$ and postseismic functions $F$ (the latter typically exponential and/or logarithmic).   $B_\perp(t)$, $R$, $\theta$, and $\delta z$ are, respectively, the perpendicular component of the interferometric baseline relative to the first date, slant range distance, incidence angle and topography error correction for the given pixel. 

Thus, given either an ensemble of interferograms or the output of SBAS (displacement vs. time), we can write the LSQ problem as 

$$ \textbf{G}\textbf{m} = \textbf{d}$$

where $\textbf{G}$ is the design matrix (constructed out of the different functional terms in Equation 2 evaluated either at the SAR image dates for SBAS output, or between the dates spanned by each pair for interferograms), $\textbf{m}$ is the vector of model parameters (the coefficients in Equation 2) and $\textbf{d}$ is the vector of observations.  For GPS time series, $\textbf{G}, \textbf{d}, \textbf{m}$ are constructed using values evaluated at single epochs corresponding to the GPS solution times, as for SBAS InSAR input. 

With this formulation, we can obtain InSAR velocity estimates and their formal uncertainties (including in areas where the expected answer is zero). 

The default InSAR velocity fit in MintPy is to estimate a mean linear velocity $(v)$ in in the equation, which we do below. 

In [ ]:
if sites[site]['use_mintpy']:
    command = 'smallbaselineApp.py ' + str(config_file) + ' --dostep velocity'
    process = subprocess.run(command, shell=True)

# load velocity file
insar_velocities,_ = readfile.read(vel_file, datasetName = 'velocity')  # read velocity file
insar_velocities = insar_velocities * 1000.  # convert velocities from m to mm

# set masked pixels to NaN
msk,_ = readfile.read(msk_file)
insar_velocities[msk == 0] = np.nan
insar_velocities[insar_velocities == 0] = np.nan

Now we plot the mean linear velocity fit. The MintPy `view` module automatically reads the temporal coherence mask `maskTempCoh.h5` and applies that to mask out pixels with unreliable velocities (white).

In [ ]:
scp_args = 'velocity.h5 velocity -v -20 20 --colormap RdBu_r --figtitle LOS_Velocity --unit mm/yr -m ' + msk_file
view.main(scp_args.split())

<div class="alert alert-info">
<b>Note :</b> 
Negative values indicates that target is moving away from the radar (i.e., Subsidence in case of vertical deformation).
Positive values indicates that target is moving towards the radar (i.e., uplift in case of vertical deformation). 
</div>

<a id='secular_co_gps'></a>
## 4.2. Find Collocated GNSS Stations

The project will have access to L2 position data for continuous GNSS stations in third-party networks such NSF’s Plate Boundary Observatory, the HVO network for Hawaii, GEONET-Japan, and GEONET-New Zealand, located in target regions for NISAR solid earth calval. Station data will be post-processed by one or more analysis centers, will be freely available, and will have latencies of several days to weeks, as is the case with positions currently produced by the NSF’s GAGE Facility and separately by the University of Nevada Reno. Networks will contain one or more areas of high-density station coverage (2~20 km nominal station spacing over 100 x 100 km or more) to support validation of L2 NISAR requirements at a wide range of length scales.

In [ ]:
# get analysis metadata from InSAR velocity file
insar_metadata = readfile.read_attribute(vel_file)
lat_step = float(insar_metadata['Y_STEP'])
lon_step = float(insar_metadata['X_STEP'])
(S,N,W,E) = ut.four_corners(insar_metadata)
start_date = insar_metadata.get('START_DATE', None)
end_date = insar_metadata.get('END_DATE', None)
start_date_gnss = dt.strptime(start_date, "%Y%m%d")
end_date_gnss = dt.strptime(end_date, "%Y%m%d")

geom_file = os.path.join(mintpy_dir, 'inputs/geometryGeo.h5')
inc_angle = readfile.read(geom_file, datasetName='incidenceAngle')[0]
inc_angle = np.nanmean(inc_angle)
az_angle = readfile.read(geom_file, datasetName='azimuthAngle')[0]
az_angle = np.nanmean(az_angle)

#Set GNSS Parameters
gps_completeness_threshold = 0.9    #0.9  #percent of data timespan with valid GNSS epochs
gps_residual_stdev_threshold = 10.  #0.03  #0.03  #max threshold standard deviation of residuals to linear GNSS fit

# search for collocated GNSS stations
site_names, site_lats, site_lons = gps.search_gps(SNWE=(S,N,W,E), start_date=start_date, end_date=end_date)
site_names = [str(stn) for stn in site_names]
print("Initial list of {} stations used in analysis:".format(len(site_names)))
print(site_names)

<a id='secular_gps_ts'></a>
## 4.3. Get GNSS Position Time Series


In [ ]:
# get daily position solutions for GNSS stations
use_stn = []  #stations to keep
bad_stn = []  #stations to toss
use_lats = [] 
use_lons = []

for counter, stn in enumerate(site_names):
    gps_obj = gps.GPS(site = stn, data_dir = os.path.join(mintpy_dir,'GPS'))
    gps_obj.open(print_msg=False)
    
    # count number of dates in time range
    dates = gps_obj.dates
    range_days = (end_date_gnss - start_date_gnss).days
    gnss_count = np.histogram(dates, bins=[start_date_gnss, end_date_gnss])
    gnss_count = int(gnss_count[0])
    
    # for this quick screening check of data quality, we use the constant incidence and azimuth angles 
    # get standard deviation of residuals to linear fit
    disp_los = ut.enu2los(gps_obj.dis_e, gps_obj.dis_n, gps_obj.dis_u, inc_angle, az_angle)
    disp_detrended = signal.detrend(disp_los)
    stn_stdv = np.std(disp_detrended)
   
    # select GNSS stations based on data completeness and scatter of residuals
    disp_detrended = signal.detrend(disp_los)
    if range_days * gps_completeness_threshold <= gnss_count:
        if stn_stdv > gps_residual_stdev_threshold:
            bad_stn.append(stn)
        else:
            use_stn.append(stn)
            use_lats.append(site_lats[counter])
            use_lons.append(site_lons[counter])
    else:
        bad_stn.append(stn)

site_names = use_stn
site_lats = use_lats
site_lons = use_lons

# [optional] manually remove additional stations
gnss_to_remove=[]

for i, gnss_site in enumerate(gnss_to_remove):
    if gnss_site in site_names:
        site_names.remove(gnss_site)
    if gnss_site not in bad_stn:
        bad_stn.append(gnss_site)

print("\nFinal list of {} stations used in analysis:".format(len(site_names)))
print(site_names)
print("List of {} stations removed from analysis".format(len(bad_stn)))
print(bad_stn)

<a id='secular_gps_los'></a>
## 4.4. Project GNSS to LOS Velocities

In [ ]:
gnss_velocities = gps.get_gps_los_obs(insar_metadata, 
                            'velocity', 
                            site_names, 
                            start_date=start_date,
                            end_date=end_date,
                            gps_comp='enu2los', 
                            redo=True)

# scale site velocities from m/yr to mm/yr
gnss_velocities *= 1000.

print('\n site   vel_los [mm/yr]')
print(np.array([site_names, gnss_velocities]).T)

<a id='secular_gps_insar'></a>
## 4.5. Re-Reference GNSS and InSAR LOS Velocities


In [ ]:
# reference GNSS stations to GNSS reference site
ref_site_ind = site_names.index(sites[site]['gps_ref_site_name'])
gnss_velocities = gnss_velocities - gnss_velocities[ref_site_ind]

# reference InSAR to GNSS reference site
ref_site_lat = float(site_lats[ref_site_ind])
ref_site_lon = float(site_lons[ref_site_ind])
ref_y, ref_x = ut.coordinate(insar_metadata).geo2radar(ref_site_lat, ref_site_lon)[:2]
if not math.isnan(insar_velocities[ref_y, ref_x]):
    insar_velocities = insar_velocities - insar_velocities[ref_y, ref_x]

# plot GNSS stations on InSAR velocity field
vmin = sites[site]['vmin']
vmax = sites[site]['vmax']
cmap = plt.get_cmap('RdBu_r')

fig, ax = plt.subplots(figsize=[18, 5.5])
cax = ax.imshow(insar_velocities, cmap=cmap, vmin=vmin, vmax=vmax, interpolation='nearest', extent=(W, E, S, N))
cbar = fig.colorbar(cax, ax=ax)
cbar.set_label('LOS velocity [mm/year]')

for lat, lon, obs in zip(site_lats, site_lons, gnss_velocities):
    color = cmap((obs - vmin)/(vmax - vmin))
    ax.scatter(lon, lat, color=color, s=8**2, edgecolors='k')
for i, label in enumerate(site_names):
     plt.annotate(label, (site_lons[i], site_lats[i]), color='black')

out_fig = os.path.abspath('vel_insar_vs_gnss.png')
fig.savefig(out_fig, bbox_inches='tight', transparent=True, dpi=300)

<a id='secular_nisar_validation'></a>
# 5. NISAR Validation Approach 1: GNSS-InSAR Direct Comparison 


<a id='secular_make_vel'></a>
## 5.1. Make Velocity Residuals at GNSS Locations


In [ ]:
#Set Parameters
pixel_radius = 5   #number of InSAR pixels to average for comparison with GNSS

#Create dictionary with the stations as the key and all their info as an array 
stn_dict = {}

#Loop over GNSS station locations
for i in range(len(site_names)): 
    # convert GNSS station lat/lon information to InSAR x/y grid
    stn_lat = site_lats[i]
    stn_lon = site_lons[i]
    x_value = round((stn_lon - W)/lon_step)
    y_value = round((stn_lat - N)/lat_step)
    
    # get velocities and residuals
    gnss_site_vel = gnss_velocities[i]
    #Caution: If you expand the radius parameter farther than the bounding grid it will break. 
    #To fix, remove the station in section 4 when the site_names list is filtered
    vel_px_rad = insar_velocities[y_value-pixel_radius:y_value+1+pixel_radius, 
                     x_value-pixel_radius:x_value+1+pixel_radius]
    insar_site_vel = np.median(vel_px_rad)
    residual = gnss_site_vel - insar_site_vel

    # populate data structure
    values = [x_value, y_value, insar_site_vel, gnss_site_vel, residual, stn_lat, stn_lon]
    stn = site_names[i]
    stn_dict[stn] = values

# extract data from structure
res_list = []
insar_site_vels = []
gnss_site_vels = []
lat_list = []
lon_list = []
for i in range(len(site_names)): 
    stn = site_names[i]
    insar_site_vels.append(stn_dict[stn][2])
    gnss_site_vels.append(stn_dict[stn][3])
    res_list.append(stn_dict[stn][4])
    lat_list.append(stn_dict[stn][5])
    lon_list.append(stn_dict[stn][6])
num_stn = len(site_names) 
print('Finish creating InSAR residuals at GNSS sites')

<a id='secular_make_velres'></a>
## 5.2. Make Double-Differenced Velocity Residuals


In [ ]:
n_gps_sites = len(site_names)
diff_res_list = []
stn_dist_list = []

# loop over stations
for i in range(n_gps_sites-1):
    stn1 = site_names[i]
    for j in range(i + 1, n_gps_sites):
        stn2 = site_names[j]

        # calculate GNSS and InSAR velocity differences between stations
        gps_vel_diff = stn_dict[stn1][3] - stn_dict[stn2][3]
        insar_vel_diff = stn_dict[stn1][2] - stn_dict[stn2][2]

        # calculate GNSS vs InSAR differences (double differences) between stations
        diff_res = gps_vel_diff - insar_vel_diff
        diff_res_list.append(diff_res)

        # get distance (km) between stations using Haversine formula
        # index 5 is lat, 6 is lon
        stn_dist = haversine_distance(stn_dict[stn1][6], stn_dict[stn1][5], stn_dict[stn2][6], stn_dict[stn2][5])
        stn_dist_list.append(stn_dist)

# Write data for statistical tests
gnss_site_dist = np.array(stn_dist_list)
double_diff_rel_measure = np.array(np.abs(diff_res_list))
ndx = np.argsort(gnss_site_dist)

# Plot data to be used below
fig, ax = plt.subplots(figsize=[11, 7])
plt.scatter(gnss_site_dist, diff_res_list, label='V_gnss - V_InSAR for station pair')
plt.axhline(secular_gnss_rqmt, color='r', linestyle='--', label='Secular rqmt')
plt.axhline(-1*secular_gnss_rqmt, color='r', linestyle='--', label='Secular rqmt')
plt.ylim(-10,10)
plt.xlim(*gnss_dist_rqmt)
plt.legend(loc='upper left')
plt.title(f"Double-Difference Residuals \n Date range {start_date}-{end_date} \n GNSS-InSAR velocities")
plt.xlabel("Distance (km)")
plt.ylabel("Double-Differenced Velocity Residual (mm/y)")
plt.show()

out_fig = os.path.abspath('secular_insar-gnss_velocity_vs_distance.png')
fig.savefig(out_fig, bbox_inches='tight', transparent=True, dpi=300)


<div class="alert alert-warning">
Final result Method 1—Successful when 68% of points below requirements line
</div>


<a id='secular_valid_method1'></a>
## 5.3. Secular Requirement Validation: Method 1


In [ ]:
# Calculate Statistics
n_bins = 10
threshold = 0.683

if findMax == 'true':
    thresh_flag = 'false'
else :
    thresh_flag = 'true'

tmp_secular_gnss_rqmt = deepcopy(secular_gnss_rqmt)
sucess_flag = thresh_flag

#  we assume that the distribution of residuals is Gaussian and 
#  that the threshold represents a 1-sigma limit within which 
#  we expect 68.3% of residuals to lie.

# define bins and data columns, the final column is the ratio as a whole
bins = np.linspace(*gnss_dist_rqmt, num=n_bins+1)
n_all = np.empty((n_bins+1), dtype=int) # number of points in each bin
n_pass = np.empty((n_bins+1), dtype=int) # number of points that pass criterion
ratio = np.empty((n_bins+1), dtype=float) # ratio of points that pass criterion

# populate bins
inds = np.digitize(gnss_site_dist, bins)
while sucess_flag == thresh_flag:
    for i in range(n_bins):
        # relative measurement
        gnss_rem = double_diff_rel_measure[inds == i+1]
        n_all[i] = np.count_nonzero(~np.isnan(gnss_rem))
        n_pass[i] = np.count_nonzero(gnss_rem < tmp_secular_gnss_rqmt)
        if n_all[i] == 0:
            ratio[i] = 1.000  # assume pass if no data fall in bin
        else:
            ratio[i] = n_pass[i]/n_all[i]

    # fill in last column
    n_all[-1] = np.sum(n_all[0:-1])
    n_pass[-1] = np.sum(n_pass[0:-1])
    ratio[-1] = n_pass[-1]/n_all[-1]

    # determine success or failure for each bin
    success_or_fail = ratio > threshold  # boolean array
    success_or_fail_str = np.array([['true' if x==True else 'false' for x in success_or_fail]])

    # build pandas table
    columns = []
    for i in range(n_bins):
        columns.append(f'{bins[i]:.2f}-{bins[i+1]:.2f}')
    columns.append('total')

    index = ['-'.join([start_date, end_date])]

    # Display Results
    n_all_pd = pd.DataFrame(n_all.reshape(1,n_bins+1),columns=columns,index=index)
    n_pass_pd = pd.DataFrame(n_pass.reshape(1,n_bins+1),columns=columns,index=index)
    ratio_pd = pd.DataFrame(ratio.reshape(1,n_bins+1),columns=columns,index=index)
    success_or_fail_pd = pd.DataFrame(success_or_fail_str.reshape(1,n_bins+1),columns=columns,index=index)

    #display(n_all_pd)  # Number of data points in each bin
    #display(n_pass_pd) # Number of data points that lie below the curve

    #Set new style for table
    s = ratio_pd.style
    s.set_table_styles([  # create internal CSS classes
        {'selector': '.true', 'props': 'background-color: #e6ffe6;'},
        {'selector': '.false', 'props': 'background-color: #ffe6e6;'},
    ], overwrite=False)
    #display(s.set_td_classes(success_or_fail_pd))  # Percentage of passing points:
    #display(success_or_fail_pd)  # Explicit pass/fail table
    sucess_flag = success_or_fail_pd.iloc[0]['total']

    if findMax == 'true' :
        tmp_secular_gnss_rqmt += 0.01
    else :
        tmp_secular_gnss_rqmt -= 0.01

display(n_all_pd)  # Number of data points in each bin
display(n_pass_pd) # Number of data points that lie below the curve
display(s.set_td_classes(success_or_fail_pd))  # Percentage of passing points:
display(success_or_fail_pd)  # Explicit pass/fail table

print(tmp_secular_gnss_rqmt, success_or_fail_pd.iloc[0]['total'])
# Overall pass/fail criterion
if success_or_fail_pd.iloc[0]['total'] == 'true':
    print("This velocity dataset passes the requirement.")
elif success_or_fail_pd.iloc[0]['total'] == 'false':
    print("This velocity dataset does not pass the requirement.")

<div class="alert alert-warning">
Final result Method 1 table by distance bin—successful when greater than 0.683
</div>


<a id='secular_nisar_validation2'></a>
# 6. NISAR Validation Approach 2: InSAR-only Structure Function

In Validation approach 2, we use a time interval and area where we assume no deformation.

In [ ]:
# plot velocity map
scp_args = 'velocity.h5 velocity -v -20 20 --colormap RdBu_r --figtitle LOS_Velocity --unit mm/yr -m ' + msk_file
view.main(scp_args.split())

<a id='secular_array_mask'></a>
## 6.1. Read Array and Mask Pixels with no Data

In [ ]:
# use the assumed non-earthquake displacement as the insar_displacment for statistics and convert to mm
insar_velocities,_ = readfile.read(vel_file, datasetName = 'velocity')  #read velocity
velStart = sites[site]['download_start_date']
insar_velocities = insar_velocities * 1000.  # convert velocities from m to mm

# set masked pixels to NaN
msk,_ = readfile.read(msk_file)
insar_velocities[msk == 0] = np.nan
insar_velocities[insar_velocities == 0] = np.nan

# display map of data after masking
cmap = plt.get_cmap('RdBu_r')

fig, ax = plt.subplots(figsize=[18, 5.5])
img1 = ax.imshow(insar_velocities, cmap=cmap, vmin=-20, vmax=20, interpolation='nearest', extent=(W, E, S, N))
ax.set_title("Secular \n Date "+velStart)
cbar1 = fig.colorbar(img1, ax=ax)
cbar1.set_label('LOS velocity [mm/year]')

<a id='secular_remove_trend'></a>
## 6.2. Randomly Sample Pixels and Pair Them Up with Option to Remove Trend

In [ ]:
sample_mode = 'profile'  # 'points' or 'profile'
# note that the 'profile' method may take significantly longer

# Collect samples using the specified method
if sample_mode in ['points']:
    X0,Y0 = load_geo(insar_metadata)
    X0_2d,Y0_2d = np.meshgrid(X0,Y0)

    insar_sample_dist, insar_rel_measure = samp_pair(X0_2d, Y0_2d, insar_velocities, num_samples=1000000)

elif sample_mode in ['profile']:
    # Sample grid setup
    length, width = int(insar_metadata['LENGTH']), int(insar_metadata['WIDTH'])
    X = np.linspace(W+lon_step, E-lon_step, width)  # longitudes
    Y = np.linspace(N+lat_step, S-lat_step, length)  # latitudes
    X_coords, Y_coords = np.meshgrid(X, Y)

    # Draw random samples from map (without replacement)
    num_samples = 20000
    
    # Retrieve profile samples
    insar_sample_dist, insar_rel_measure = profile_samples(\
                    x=X_coords.reshape(-1,1),
                    y=Y_coords.reshape(-1,1),
                    data=insar_velocities,
                    metadata=insar_metadata,
                    len_rqmt=insar_dist_rqmt,
                    num_samples=num_samples)

print('Finished sampling')

In [ ]:
fig, ax = plt.subplots(figsize=[18, 5.5])
img1 = ax.hist(insar_sample_dist, bins=100)
ax.set_title("Histogram of distance \n Secular Date {:s} - {:s}".format(start_date, end_date))
ax.set_xlabel(r'Distance ($km$)')
ax.set_ylabel('Frequency')
ax.set_xlim(*insar_dist_rqmt)
    
fig, ax = plt.subplots(figsize=[18, 5.5])
img1 = ax.hist(insar_rel_measure, bins=100)
ax.set_title("Histogram of Relative Measurement \n Secular Date {:s} - {:s}".format(start_date, end_date))
ax.set_xlabel(r'Relative Measurement ($mm/year$)')
ax.set_ylabel('Frequency')

<a id='secular_M2ampvsdist2'></a>
## 6.3. Amplitude vs. Distance of Relative Measurements (pair differences)

In [ ]:
fig, ax = plt.subplots(figsize=[18, 7.5])
plt.axhline(secular_insar_rqmt, color='r', linestyle='--', label='Secular rqmt')
ax.scatter(insar_sample_dist, insar_rel_measure, s=1, alpha=0.25, label='Relative velocity for pixel pair')
ax.set_title(f"Method 2: Relative Velocity Measurements between Pixel Pairs and Requirement Curve vs. Distance \n {site} Secular Date range {start_date}-{end_date}" )
ax.set_ylabel(r'Relative Velocity Measurement ($mm/year$)')
ax.set_xlabel('Distance (km)')
ax.set_xlim(*insar_dist_rqmt)
plt.legend(loc='upper left')


plt.show()

out_fig = os.path.abspath('secular_insar-only_vs_distance_'+site+'_date'+velStart+'.png')
fig.savefig(out_fig, bbox_inches='tight', transparent=True, dpi=300)

<div class="alert alert-warning">
Final result Method 2—
    68% of points below the requirements line is success
</div>


<a id='secular_M2RelMeasTable'></a>
## 6.4. Bin Sample Pairs by Distance Bin and Calculate Statistics

In [ ]:
# Calculate Statistics
n_bins = 10
threshold = 0.683 

if findMax == 'true':
    thresh_flag = 'false'
else :
    thresh_flag = 'true'

tmp_secular_insar_rqmt = deepcopy(secular_insar_rqmt)
sucess_flag = thresh_flag

#  we assume that the distribution of residuals is Gaussian and 
#  that the threshold represents a 1-sigma limit within which 
#  we expect 68.3% of residuals to lie.

# define bins and data columns, the final column is the ratio as a whole
bins = np.linspace(*insar_dist_rqmt, num=n_bins+1)
n_all = np.empty((n_bins+1), dtype=int) # number of points in each bin
n_pass = np.empty((n_bins+1), dtype=int) # number of points that pass criterion
ratio = np.empty((n_bins+1), dtype=float) # ratio of points that pass criterion

# populate bins
inds = np.digitize(insar_sample_dist, bins)
while sucess_flag == thresh_flag:
    for i in range(n_bins):
        # relative measurement
        insar_rem = insar_rel_measure[inds == i+1]
        n_all[i] = np.count_nonzero(~np.isnan(insar_rem))
        n_pass[i] = np.count_nonzero(insar_rem < tmp_secular_insar_rqmt)
        if n_all[i] == 0:
            ratio[i] = 1.000  # assume pass if no data fall in bin
        else:
            ratio[i] = n_pass[i]/n_all[i]

    # fill in last column
    n_all[-1] = np.sum(n_all[0:-1])
    n_pass[-1] = np.sum(n_pass[0:-1])
    ratio[-1] = n_pass[-1]/n_all[-1]

    # determine success or failure for each bin
    success_or_fail = ratio > threshold  # boolean array
    success_or_fail_str = np.array([['true' if x==True else 'false' for x in success_or_fail]])

    # build pandas table
    columns = []
    for i in range(n_bins):
        columns.append(f'{bins[i]:.2f}-{bins[i+1]:.2f}')
    columns.append('total')

    index = ['-'.join([start_date, end_date])]
        
    # Display Results
    n_all_pd = pd.DataFrame(n_all.reshape(1,n_bins+1),columns=columns,index=index)
    n_pass_pd = pd.DataFrame(n_pass.reshape(1,n_bins+1),columns=columns,index=index)
    ratio_pd = pd.DataFrame(ratio.reshape(1,n_bins+1),columns=columns,index=index)
    success_or_fail_pd = pd.DataFrame(success_or_fail_str.reshape(1,n_bins+1),columns=columns,index=index)

    #display(n_all_pd)  # Number of data points in each bin
    #display(n_pass_pd) # Number of data points that lie below the curve

    #Set new style for table
    s = ratio_pd.style
    s.set_table_styles([  # create internal CSS classes
        {'selector': '.true', 'props': 'background-color: #e6ffe6;'},
        {'selector': '.false', 'props': 'background-color: #ffe6e6;'},
    ], overwrite=False)
    #display(s.set_td_classes(success_or_fail_pd))  # Percentage of passing points:
    #display(success_or_fail_pd)  # Explicit pass/fail table
    sucess_flag = success_or_fail_pd.iloc[0]['total']
    if findMax == 'true' :
        tmp_secular_insar_rqmt += 0.01
    else :
        tmp_secular_insar_rqmt -= 0.01

display(n_all_pd)  # Number of data points in each bin
display(n_pass_pd) # Number of data points that lie below the curve
display(s.set_td_classes(success_or_fail_pd))  # Percentage of passing points:
display(success_or_fail_pd)  # Explicit pass/fail table

print(tmp_secular_insar_rqmt, success_or_fail_pd.iloc[0]['total'])
# Overall pass/fail criterion
if success_or_fail_pd.iloc[0]['total'] == 'true':
    print("This velocity dataset passes the requirement.")
elif success_or_fail_pd.iloc[0]['total'] == 'false':
    print("This velocity dataset does not pass the requirement.")

<div class="alert alert-warning">
Final result Method 2 table of distance bins—
    68% (0.683) of points below the requirements line is success
</div>


<a id='secular_appendix1'></a>
# Appendix: Supplementary Comparisons and Plots

<a id='secular_compare_raw'></a>
## A.1. Compare Raw Velocities

In [ ]:
vmin, vmax = -25, 25
plt.figure(figsize=(11,7))
plt.hist(insar_site_vels, range=[vmin, vmax], bins=50, color="green", edgecolor='grey', label='V_InSAR')
plt.hist(gnss_site_vels, range=[vmin, vmax], bins=50, color="orange", edgecolor='grey', label='V_gnss', alpha=0.5)
plt.legend(loc='upper right')
plt.title(f"Velocities \n Date range {start_date}-{end_date} \n Reference stn: {sites[site]['gps_ref_site_name']} \n Number of stations used: {num_stn}")
plt.xlabel('LOS Velocity (mm/year)')
plt.ylabel('N Stations')
plt.ylim(0,20)
plt.show()

<a id='secular_plot_vel'></a>
## A.2. Plot Velocity Residuals


In [ ]:
vmin, vmax = -10, 10
plt.figure(figsize=(11,7))
plt.hist(res_list, bins = 40, range=[vmin,vmax], edgecolor='grey', color="darkblue", linewidth=1, label='V_gnss - V_InSAR (area average)')
plt.legend(loc='upper right')
plt.title(f"Residuals \n Date range {start_date}-{end_date} \n Reference stn: {sites[site]['gps_ref_site_name']} \n Number of stations used: {num_stn}")
plt.xlabel('Velocity Residual (mm/year)')
plt.ylabel('N Stations')
plt.show()

<a id='secular_plot_velres'></a>
## A.3. Plot Double Difference Residuals

In [ ]:
plt.figure(figsize=(11,7))
plt.hist(diff_res_list, range = [vmin, vmax],bins = 40, color = "darkblue",edgecolor='grey',label='V_gnss_(s1-s2) - V_InSAR_(s1-s2)')
plt.legend(loc='upper right')
plt.title(f"Difference Residualts \n Date range {start_date}-{end_date} \n Reference stn: {sites[site]['gps_ref_site_name']} \n Number of stations used: {num_stn}")
plt.xlabel('Double Differenced Velocity Residual (mm/year)')
plt.ylabel('N Stations')
plt.show()

<a id='secular_appendix_gps'></a>
## A.4. GNSS Timeseries Plots


In [ ]:
# grab the time-series file used for time function estimation given the template setup
template = readfile.read_template(os.path.join(mintpy_dir, 'smallbaselineApp.cfg'))
template = ut.check_template_auto_value(template)
insar_ts_file = TimeSeriesAnalysis.get_timeseries_filename(template, mintpy_dir)['velocity']['input']

# read the time-series file
insar_ts, atr = readfile.read(insar_ts_file, datasetName='timeseries')
mask = readfile.read(os.path.join(mintpy_dir, 'maskTempCoh.h5'))[0]
print(f'reading timeseries from file: {insar_ts_file}')

# Get date list
date_list = timeseries(insar_ts_file).get_date_list()
num_date = len(date_list)
date0, date1 = date_list[0], date_list[-1]
insar_dates = ptime.date_list2vector(date_list)[0]

# spatial reference
coord = ut.coordinate(atr)
ref_site = sites[site]['gps_ref_site_name']
ref_gnss_obj = gps.GPS(site=ref_site, data_dir=mintpy_dir/'GPS')
ref_lat, ref_lon = ref_gnss_obj.get_stat_lat_lon()
ref_y, ref_x = coord.geo2radar(ref_lat, ref_lon)[:2]
if not mask[ref_y, ref_x]:
    raise ValueError(f'Given reference GNSS site ({ref_site}) is in mask-out unrelible region in InSAR! Change to a different site.')
ref_insar_dis = insar_ts[:, ref_y, ref_x]

# Plot displacements and velocity timeseries at GNSS station locations
num_site = len(site_names)
prog_bar = ptime.progressBar(maxValue=num_site)
for i, site_name in enumerate(site_names):
    prog_bar.update(i+1, suffix=f'{site_name} {i+1}/{num_site}')

    ## read data
    # read GNSS
    gnss_obj = gps.GPS(site=site_name, data_dir=mintpy_dir/'GPS')
    gnss_dates, gnss_dis, _, gnss_lalo = gnss_obj.read_gps_los_displacement(atr, start_date=date0, end_date=date1, ref_site=ref_site)[:4]
    # shift GNSS to zero-mean in time [for plotting purpose]
    gnss_dis -= np.nanmedian(gnss_dis)

    # read InSAR
    y, x = coord.geo2radar(gnss_lalo[0], gnss_lalo[1])[:2]
    insar_dis = insar_ts[:, y, x] - ref_insar_dis
    # apply a constant shift in time to fit InSAR to GNSS
    comm_dates = sorted(list(set(gnss_dates) & set(insar_dates)))
    if comm_dates:
        insar_flag = [x in comm_dates for x in insar_dates]
        gnss_flag = [x in comm_dates for x in gnss_dates]
        insar_dis -= np.nanmedian(insar_dis[insar_flag] - gnss_dis[gnss_flag])

    ## plot figure
    if gnss_dis.size > 0 and np.any(~np.isnan(insar_dis)):
        fig, ax = plt.subplots(figsize=(12, 3))
        ax.axhline(color='grey',linestyle='dashed', linewidth=2)
        ax.scatter(gnss_dates, gnss_dis*100, s=2**2, label="GNSS Daily Positions")
        ax.scatter(insar_dates, insar_dis*100, label="InSAR Positions")
        # axis format
        ax.set_title(f"Station Name: {site_name}") 
        ax.set_ylabel('LOS displacement [cm]')
        ax.legend()
prog_bar.close()
plt.show()